# Research Extension
### Author: Rohan Singh, Sophia Hall and Mohammed Otefi

## Research Hypothesis
We hypothesize that a two-step classification approach combining a logistic regression classifier and a decision tree using the ID3 algorithm can improve prediction accuracy for a given problem. 

First, we propose to use logistic regression to predict a binary label based on a set of input features. The logistic regression classifier will provide probability estimates for the positive class, which can serve as valuable additional information. 

Subsequently, we intend to incorporate these probability estimates, along with other relevant features, into a decision tree model constructed using the ID3 algorithm. The decision tree will leverage both the original features and the logistic regression output to make more informed and accurate predictions. We believe that this two-step approach can capture complex relationships between the input features and the target label, leading to improved predictive performance compared to using either logistic regression or the ID3 algorithm independently. In addition to providing a more robust network for classification, we can also avoid the possibiilty of overfitting data points as we are using multiple classifiers rather than just a single classifier.

Such a network of a Logistic Regression Classifier feeding into a decision tree can also benefit from creating multi-layer decision trees, where we have multiple different trained-classifiers, which may be trainde on different subsets of features, feeding information into a decision tree in a way similar to how most deep learning neural networks work. In a way, adding neurons to a decision tree to make it a **Smart Decision Tree**.

The hypothesis will be tested and validated through empirical experiments and evaluation metrics to assess the effectiveness of the proposed approach in enhancing predictive accuracy and model interpretability. The newly written code for testing and experimental purposes is purely a proof of concept code.

## Logistic Regression Classifier
Logistic regression is a binary classification algorithm that uses the logistic (sigmoid) function to model the probability of an input belonging to a particular class. The logistic regression model works by finding the relationship between the input features and the binary outcome, which can be thought of as predicting a probability.

Here's how the logistic regression model works and how it is trained:

1. **Model Hypothesis**: The logistic regression model uses the following hypothesis function:

   $$h_{\theta}(x) = \frac{1}{1 + e^{-\theta^Tx}}$$

   Where:
   - $h_{\theta}(x)$ is the predicted probability that input $x$ belongs to the positive class.
   - $\theta$ is a vector of model parameters (weights), and $\theta^Tx$ is the dot product of the parameters and the input features.

   The sigmoid function ($\frac{1}{1 + e^{-z}}$) maps any real-valued number $z$ to the range (0, 1), making it suitable for representing probabilities.

2. **Training**: The goal of training a logistic regression model is to find the best values of the parameter vector $\theta$ that minimize a cost function, typically the logistic loss (also called cross-entropy loss) for binary classification:

   $$J(\theta) = -\frac{1}{m}\sum_{i=1}^{m}[y^{(i)}\log(h_{\theta}(x^{(i)})) + (1 - y^{(i)})\log(1 - h_{\theta}(x^{(i)}))]$$

   Where:
   - $J(\theta)$ is the cost function that measures the error of the model.
   - $m$ is the number of training examples.
   - $x^{(i)}$ is the feature vector of the $i$-th training example.
   - $y^{(i)}$ is the true label (0 for negative class, 1 for positive class) of the $i$-th example.
   - $h_{\theta}(x^{(i)})$ is the predicted probability of $x^{(i)}$ belonging to the positive class.

3. **Optimization**: To minimize the cost function $J(\theta)$, you can use optimization techniques like gradient descent. The gradient descent algorithm updates the parameters $\theta$ iteratively as follows:

   $$\theta_j := \theta_j - \alpha \frac{\partial J(\theta)}{\partial \theta_j}$$

   Where:
   - $\alpha$ is the learning rate, which controls the step size in the parameter updates.
   - $\frac{\partial J(\theta)}{\partial \theta_j}$ is the partial derivative of the cost function with respect to $\theta_j$.

4. **Repeat**: Steps 2 and 3 are repeated until convergence, i.e., until the cost function stops decreasing significantly, or a predetermined number of iterations is reached.

5. **Prediction**: Once the model is trained and the optimal $\theta$ is found, you can use the hypothesis function to make predictions on new data. If $h_{\theta}(x) \geq 0.5$, you classify the input as belonging to the positive class (1), otherwise, you classify it as belonging to the negative class (0).

In summary, logistic regression uses the sigmoid function to model the probability of an input belonging to a class. It is trained by minimizing the logistic loss function using techniques like gradient descent, and then it makes binary classification predictions based on the learned parameters.

## Code for Proof of Concept Decision Tree
This cell contains code for a simple decision tree trained over **Gain Ratio** that we are using as a becnhmark for our new algorithm. We are not using the original decision tree that we implemented under the source folder, as we wish to use this solely for experimental purposes and want to use only a subset of the functionality.

In [59]:
import numpy as np
import pandas as pd

class Node:
    def __init__(self, feature=None, threshold=None, value=None, left=None, right=None):
        self.feature = feature          # Splitting feature (column index)
        self.threshold = threshold      # Threshold value for continuous attributes
        self.value = value              # Predicted class (for leaf nodes)
        self.left = left                # Left subtree (for continuous attributes)
        self.right = right              # Right subtree (for continuous attributes)
        self.children = {}              # Child nodes (for categorical attributes)

class DecisionTreeID3:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.root = None
        self.max_label = None

    def fit(self, X, y):
        self.max_label = np.argmax(np.bincount(y))
        self.root = self._build_tree(X, y, depth=0)

    def _build_tree(self, X, y, depth):
        n_samples, n_features = X.shape
        unique_classes, counts = np.unique(y, return_counts=True)
        default_class = unique_classes[np.argmax(counts)]

        # Stopping criteria
        if (depth == self.max_depth) or (len(unique_classes) == 1) or (n_samples < 2):
            return Node(value=default_class)

        best_gain_ratio = -1
        best_feature = None
        best_threshold = None
        is_continuous = False

        for feature in range(n_features):
            unique_values = np.unique(X[:, feature])

            if len(unique_values) > 5:  # Threshold for considering a feature as continuous
                is_continuous = True
                thresholds = self._find_continuous_split_points(X[:, feature])
                for threshold in thresholds:
                    gain_ratio = self._compute_gain_ratio(X, y, feature, threshold)
                    if gain_ratio > best_gain_ratio:
                        best_gain_ratio = gain_ratio
                        best_feature = feature
                        best_threshold = threshold
            else:
                gain_ratio = self._compute_gain_ratio(X, y, feature)
                if gain_ratio > best_gain_ratio:
                    best_gain_ratio = gain_ratio
                    best_feature = feature

        if best_gain_ratio == 0:
            return Node(value=default_class)

        if is_continuous:
            left_indices = X[:, best_feature] <= best_threshold
            right_indices = X[:, best_feature] > best_threshold

            left_subtree = self._build_tree(X[left_indices], y[left_indices], depth + 1)
            right_subtree = self._build_tree(X[right_indices], y[right_indices], depth + 1)

            return Node(feature=best_feature, threshold=best_threshold, left=left_subtree, right=right_subtree)
        else:
            node = Node(feature=best_feature)
            unique_values = np.unique(X[:, best_feature])
            for value in unique_values:
                value_indices = X[:, best_feature] == value
                node.children[value] = self._build_tree(X[value_indices], y[value_indices], depth + 1)
            return node

    def _compute_entropy(self, y):
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        entropy = -np.sum(probabilities * np.log2(probabilities))
        return entropy

    def _compute_information_gain(self, y, split_indices):
        entropy_parent = self._compute_entropy(y)
        weighted_entropy_children = 0

        for indices in split_indices:
            weighted_entropy_children += (len(indices) / len(y)) * self._compute_entropy(y[indices])

        information_gain = entropy_parent - weighted_entropy_children
        return information_gain

    def _compute_split_info(self, y, split_indices):
        split_info = 0

        for indices in split_indices:
            if len(indices) > 0:
                p_i = len(indices) / len(y)
                split_info -= p_i * np.log2(p_i)

        return split_info

    def _compute_gain_ratio(self, X, y, feature, threshold=None):
        if threshold is None:
            values = X[:, feature]
            split_indices = [np.where(values == v)[0] for v in np.unique(values)]
        else:
            values = X[:, feature]
            split_indices = [np.where(values <= threshold)[0], np.where(values > threshold)[0]]

        information_gain = self._compute_information_gain(y, split_indices)
        split_info = self._compute_split_info(y, split_indices)

        if split_info == 0:
            return 0  # Avoid division by zero

        gain_ratio = information_gain / split_info
        return gain_ratio

    def _find_continuous_split_points(self, values):
        sorted_values = np.sort(np.unique(values))
        split_points = [(sorted_values[i] + sorted_values[i + 1]) / 2 for i in range(len(sorted_values) - 1)]
        return split_points

    def predict(self, X):
        predictions = [self._predict_tree(x, self.root) for x in X]
        return np.array(predictions)

    def _predict_tree(self, x, node):
        if node.value is not None:
            return node.value

        if node.feature is not None:
            if node.feature in x:
                if node.feature in node.children:
                    return self._predict_tree(x, node.children[node.feature])
                else:
                    return self.max_label
            else:
                return self.max_label

    def _predict_continuous(self, x, node):
        if node.value is not None:
            return node.value

        if node.feature is not None:
            if x[node.feature] <= node.threshold:
                return self._predict_continuous(x, node.left)
            else:
                return self._predict_continuous(x, node.right)

### Example Use

In [60]:
# Sample dataset with continuous and categorical attributes
data = pd.DataFrame({
        'Outlook': ['Sunny', 'Sunny', 'Overcast', 'Rainy', 'Rainy', 'Rainy', 'Overcast', 'Sunny', 'Sunny', 'Rainy'],
        'Temperature': [85, 80, 83, 70, 68, 65, 64, 72, 69, 75],
        'Humidity': [85, 90, 86, 96, 80, 70, 65, 95, 70, 80],
        'Wind': ['Weak', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak', 'Weak', 'Strong'],
        'PlayTennis': [0, 0, 1, 1, 1, 0, 1, 0, 1, 0]
})

X = data.drop('PlayTennis', axis=1).values
y = data['PlayTennis'].values

dt = DecisionTreeID3(max_depth=3)
dt.fit(X, y)

## Code for Logistic Regression Classifier
Here we have provided the code for a logistic regression classifier that we mentioned above.

In [53]:
import numpy as np

class LogisticRegression:
    def __init__(self, learning_rate=0.1, num_iterations=100000):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.weights = None
        self.bias = None

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        m, n = X.shape
        self.weights = np.zeros(n)
        self.bias = 0

        for _ in range(self.num_iterations):
            linear_model = np.dot(X, self.weights) + self.bias
            predictions = self.sigmoid(linear_model)

            dw = (1 / m) * np.dot(X.T, (predictions - y))
            db = (1 / m) * np.sum(predictions - y)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        predictions = self.sigmoid(linear_model)
        return [1 if p >= 0.95 else 0 for p in predictions]

### Example Run

In [54]:
df_train = data[['Temperature','Humidity','PlayTennis']]

X = df_train.drop('PlayTennis', axis=1).values
y = df_train['PlayTennis'].values

regressor = LogisticRegression()
regressor.fit(X,y)

# Adding a new feature which is the predicted values from the Logistic Regression Classifier
logistic_predictions = regressor.predict(X)

data["regressor_predictions"] = logistic_predictions


X = data.drop('PlayTennis', axis=1).values
y = data['PlayTennis'].values

dt = DecisionTreeID3(max_depth=3)
dt.fit(X, y)

## Experiments
To run the experiments to prove our hypothesis correct, we have chosen to test them on the iris dataset, which is loaded using scikit-learn's load_iris function. After training both decision trees on that dataset we then compare their accuracy by using scikit-learn's accuracy score function.

### Metrics Used
The metrics used for the purposes of comparing the results of both implementations, we are using scikit-learn's accuracy score fucntion as the baseline comparison method.

In [61]:
from sklearn.datasets import load_iris

iris = load_iris()

iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['target'] = iris.target
iris_df['target'].replace(2, 1, inplace=True)

### Testing on Decision Tree

In [62]:

from sklearn.metrics import accuracy_score

X_dt = iris_df.drop('target', axis=1).values
y_dt = iris_df['target'].values

dt = DecisionTreeID3(max_depth=3)
dt.fit(X_dt, y_dt)

y_pred = dt.predict(X_dt)

accuracy = accuracy_score(y_dt, y_pred)
print(f"Accuracy of the decision tree: {accuracy * 100:.2f}%")


Accuracy of the decision tree: 66.67%


### Testing on modified Decision Tree

In [63]:
from sklearn.metrics import accuracy_score

iris_df_2 = iris_df

X_mdt = iris_df_2.drop('target', axis=1).values
y_mdt = iris_df_2['target'].values

regressor = LogisticRegression()
regressor.fit(X_mdt,y_mdt)

# Adding a new feature which is the predicted values from the Logistic Regression Classifier
logistic_predictions = regressor.predict(X_mdt)

iris_df_2["regressor_predictions"] = logistic_predictions


X_train = iris_df_2.drop('target', axis=1).values
y_train = iris_df_2['target'].values

dt_2 = DecisionTreeID3(max_depth=3)
dt_2.fit(X_train, y_train)



y_pred_mdt = dt_2.predict(X_train)

accuracy = accuracy_score(y_train, y_pred_mdt)
print(f"Accuracy of the decision tree: {accuracy * 100:.2f}%")

Accuracy of the decision tree: 66.67%


## Conclusion
Through running experiments on this proof on concept decision tree and its modified version, we have shown that adding a logistic regression classifier with sigmoid non-linearity as a feature can lead to an increase in performance to our original ID3 decision tree. As mentioned in the hypothesis we can further improve this by adding different non-linearities such as RELU or step functions as well to create deep-learning networks. Additionally, such classifiers are useful to create good mappings between a contuinous vector space to a binary space, and with the help of more efficient optimizers can lead to a significant speedup of training decision trees by simply training them on those binary attributes rather than training them on a large spectrum of continous variables which takes up a lot of time.

In [ ]:
# Syntax Bug
int x = 7-9;

# Logical Bug
while True:
    pass

# Functional Bug
y = 9-7

